In [47]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

In [48]:
start = [0, 100, 200,300]
financial_data = {
    'symbol':[],
    'name': [],
    'price': [],
    'change': [],
    'per_change': [],
    'volume': [],
    'avg_volume': [],
    'market_cap': [],
    'p_e_ratio': [],
    'year_percent_change': []
}
for i in start:
    URL = f"https://finance.yahoo.com/markets/stocks/most-active/?start={i}&count=100"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    
    webpage = requests.get(URL, headers=headers)
    soup = BeautifulSoup(webpage.content, "html.parser")
    
    container = soup.find('tbody', class_="body yf-hhhli1")
    # print(container)
    
    
    
    # finding names
    for i in container.find_all('div', {'class': 'yf-362rys'}):
        financial_data['name'].append(i.get_text(strip=True))
        
    # finding prices
    for i in container.find_all('fin-streamer', {'data-field': 'regularMarketPrice'}):
        financial_data['price'].append(float(i.get_text(strip=True)))
        
    for row in container.find_all('tr', class_=['row', 'false', 'yf-hhhli1']):
        components = row.find_all('td', class_='cell tw-h-10 tw-py-0 yf-hhhli1') 
        financial_data['symbol'].append(components[0].get_text(strip=True))
        financial_data['change'].append(components[3].get_text(strip=True))
        financial_data['per_change'].append(components[4].get_text(strip=True))
        financial_data['volume'].append(components[5].get_text(strip=True))
        financial_data['avg_volume'].append(components[6].get_text(strip=True))
        financial_data['market_cap'].append(components[7].get_text(strip=True))
        financial_data['p_e_ratio'].append(components[8].get_text(strip=True))
        financial_data['year_percent_change'].append(components[9].get_text(strip=True))

In [49]:
df = pd.DataFrame(financial_data)
df

,symbol,name,price,change,per_change,volume,avg_volume,market_cap,p_e_ratio,year_percent_change
0,NVDA,NVIDIA Corporation,115.74,+6.99,+6.43%,320.76M,268.911M,2.824T,39.37,19.66%
1,TSLA,"Tesla, Inc.",248.09,+17.51,+7.59%,139.782M,88.97M,797.986B,122.21,36.05%
2,F,Ford Motor Company,9.59,-0.10,-1.03%,129.745M,84.276M,38.009B,6.57,-21.67%
3,LCID,"Lucid Group, Inc.",2.16,+0.0700,+3.35%,117.783M,93.18M,6.548B,-,-25.36%
4,PLTR,Palantir Technologies Inc.,83.65,+5.60,+7.17%,115.866M,98.585M,196.189B,440.26,212.20%
...,...,...,...,...,...,...,...,...,...,...
300,CPNG,"Coupang, Inc.",22.27,+0.29,+1.32%,8.069M,7.893M,40.208B,278.38,12.03%
301,IBN,ICICI Bank Limited,28.79,+0.27,+0.95%,5.022M,3.989M,101.689B,18.34,9.44%
302,YMM,Full Truck Alliance Co. Ltd.,12.69,+0.11,+0.87%,5.008M,7.909M,13.271B,30.21,61.08%
303,ETN,Eaton Corporation plc,292.04,+9.73,+3.45%,5.022M,3.351M,114.48B,30.71,-5.97%


In [50]:
df['per_change'] = df['per_change'].str.replace("%", "")
df['per_change'] = df['per_change'].astype(float, errors='ignore')
df['change'] = df['change'].astype(float)

In [51]:
df['p_e_ratio'] = df['p_e_ratio'].replace("-", pd.NA)
df['p_e_ratio'] = pd.to_numeric(df['p_e_ratio'], errors='coerce')

In [72]:
df['year_percent_change'] = df['year_percent_change'].str.replace("%", "")
df['year_percent_change'] = df['year_percent_change'].astype(float, errors='ignore')

In [59]:
mp = {'K':' * 10**3', 'M':' * 10**6', 'B':' * 10**9', 'T':' * 10**12', 'q':' * 10**15', 'Q':' * 10**15'}
df['volume'] = df['volume'].replace(mp.keys(), mp.values(), regex=True)
df['volume'] = df.apply(lambda x: eval(x.volume), axis=1)

In [62]:
df['avg_volume'] = df['avg_volume'].replace(mp.keys(), mp.values(), regex=True)
df['avg_volume'] = df.apply(lambda x: eval(x.avg_volume), axis=1)

In [67]:
df['market_cap'] = df['market_cap'].replace(mp.keys(), mp.values(), regex=True)

In [68]:
df['market_cap'] = df.apply(lambda x: eval(x.market_cap), axis=1)

In [74]:
df.to_csv('yahoo_finance.csv')

In [75]:
import sqlalchemy as sal
engine = sal.create_engine('mssql://Kabir-Khan-PC/master?driver=ODBC+DRIVER+17+FOR+SQL+SERVER')
conn = engine.connect()

In [77]:
df.to_sql('yahoo_finance', con=conn, index=False, if_exists='append')

96

In [78]:
df

,symbol,name,price,change,per_change,volume,avg_volume,market_cap,p_e_ratio,year_percent_change
0,NVDA,NVIDIA Corporation,115.74,6.99,6.43,320760000.0,268911000.0,2.824000e+12,39.37,19.66
1,TSLA,"Tesla, Inc.",248.09,17.51,7.59,139782000.0,88970000.0,7.979860e+11,122.21,36.05
2,F,Ford Motor Company,9.59,-0.10,-1.03,129745000.0,84276000.0,3.800900e+10,6.57,-21.67
3,LCID,"Lucid Group, Inc.",2.16,0.07,3.35,117783000.0,93180000.0,6.548000e+09,NaN,-25.36
4,PLTR,Palantir Technologies Inc.,83.65,5.60,7.17,115866000.0,98585000.0,1.961890e+11,440.26,212.20
...,...,...,...,...,...,...,...,...,...,...
300,CPNG,"Coupang, Inc.",22.27,0.29,1.32,8069000.0,7893000.0,4.020800e+10,278.38,12.03
301,IBN,ICICI Bank Limited,28.79,0.27,0.95,5022000.0,3989000.0,1.016890e+11,18.34,9.44
302,YMM,Full Truck Alliance Co. Ltd.,12.69,0.11,0.87,5008000.0,7909000.0,1.327100e+10,30.21,61.08
303,ETN,Eaton Corporation plc,292.04,9.73,3.45,5022000.0,3351000.0,1.144800e+11,30.71,-5.97
